In [ ]:
import torch
import torch.nn as nn
import random
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 96, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Conv2d(96, 192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 10, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(6, 6),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x
    
from utils import get_parameters

shape_2d = [(96, 27), (1, 96), (96, 864), (1, 96), (96, 864), (1, 96), (192, 864), (1, 192), (192, 1728), (1, 192), (192, 1728), (1, 192), (192, 1728), (1, 192), (192, 192), (1, 192), (10, 192), (1, 10)]
p_value = [13, 1, 48, 1, 48, 1, 96, 1, 96, 1, 96, 1, 96, 1, 96, 1, 5, 1]
rand = [random.randint(1, 9999) for _ in range(len(shape_2d))]
U = [
    torch.randn(
        (size[0], p),
        generator=torch.Generator().manual_seed(n),
        requires_grad=False,
    ).to("cuda")
    for size, p, n in zip(shape_2d, p_value, rand)
]
W = [val.to("cuda") for _, val in net.state_dict().items()]
V = []
for u, w, size in zip(U, W, shape_2d):
    pinverse = torch.mm(torch.linalg.inv(torch.mm(u.T, u)), u.T)
    w = w.view(size)
    V.append(torch.mm(pinverse, w))